In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft "trl<0.15.0" triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 44.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.3.8 requires tyro, which is not installed.
unsloth-zoo 2025.3.8 requires protobuf<4.0.0,

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from unsloth import FastLanguageModel
import torch

<ipython-input-3-d1cf68fe37fa>:2: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# Tải mô hình và tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/ielts_feedback_outputs_4/checkpoint-3000",
    torch_dtype=torch.float16
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/ielts_feedback_outputs_4/checkpoint-3000")

config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

In [ ]:
PROMPT = """
- Prompt:
{}

- Essay:
{}
"""

In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/ielts dataset/updated_evaluations.csv')

In [ ]:
data.head()

,prompt,essay,evaluation
0,Interviews form the basic criteria for most la...,It is believed by some experts that the tradit...,- Task Achievement :\n* Expert Explanation:\n+...
1,Interviews form the basic selecting criteria f...,Nowadays numerous huge firms allocate an inter...,- Task Achievement :\n* Expert Explanation:\n+...
2,Interview form the basic selection criteria fo...,The interview section is the most vital part o...,- Task Achievement:\n* Expert Explanation:\n+ ...
3,Interviews form the basic selection criteria f...,It is argued that the best method to recruit e...,- Task Achievement:\n* Expert Explanation:\n+ ...
4,Interviews from the basic selecting criteria f...,Nowadays many companies conduct interviews bef...,- Task Achievement:\n* Expert Explanation:\n+ ...


In [ ]:
len(data)

9831

Tạo embedding

In [ ]:
PROMPT = """
- Prompt:
{}

- Essay:
{}
"""

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

def get_embedding(prompt,essay, pooling='last'):
    """
    Hàm nhận vào một văn bản và trả về embedding sau khi pooling hidden states.
    Các tùy chọn pooling: 'mean' (trung bình), 'cls' (lấy token đầu tiên) hoặc 'last' (token cuối cùng)
    """
    # Tokenize văn bản
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to("cuda") for k, v in inputs.items()}
    """

    inputs = tokenizer(
            [
                PROMPT.format(
                    prompt,
                    essay
                )
            ], return_tensors="pt").to("cuda")

    # Lấy các hidden state từ mô hình
    # Lưu ý: Một số mô hình yêu cầu thiết lập config để trả về hidden_states, nếu không thì bạn cần set config.output_hidden_states=True.
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True, return_dict=True)

    # Giả sử sử dụng hidden states từ layer cuối cùng
    hidden_states = outputs.hidden_states[-1]  # shape: (batch_size, seq_len, hidden_size)

    # Thực hiện pooling: ví dụ trung bình các hidden states theo chiều seq_len
    if pooling == 'mean':
        embedding = hidden_states.mean(dim=1)
    elif pooling == 'cls':
        # Nếu mô hình có token [CLS] ở vị trí đầu tiên
        embedding = hidden_states[:, 0, :]
    elif pooling == 'last':
        # Lấy token cuối cùng
        embedding = hidden_states[:, -1, :]
    else:
        raise ValueError("Chọn pooling là 'mean', 'cls' hoặc 'last'.")

    return embedding

# Ví dụ về văn bản cho prompt và essay
prompt_text = data['prompt'][0]
essay_text = data['essay'][0]

# Lấy embedding cho từng phần
embedding = get_embedding(prompt_text,essay_text)

embedding = torch.tensor(embedding, dtype=torch.float32)

# In ra kích thước các embedding
print("Kích thước embedding: ", embedding.shape)

Kích thước embedding:  torch.Size([1, 3072])


<ipython-input-9-cd28a338fc16>:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embedding = torch.tensor(embedding, dtype=torch.float32)


In [ ]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from ast import literal_eval  # Nếu cần chuyển chuỗi thành danh sách khi load lại từ CSV

# Đảm bảo tiến trình tqdm hiển thị cho pandas
tqdm.pandas()

# Tắt chế độ hiển thị rút gọn của NumPy (tuỳ chọn)
np.set_printoptions(threshold=np.inf, linewidth=3000)

# Hàm lấy embedding từ một đoạn văn bản
def get_embedding(prompt,essay, pooling='last'):
    """
    Hàm nhận vào một văn bản và trả về embedding sau khi pooling hidden states.
    Các tùy chọn pooling: 'mean' (trung bình), 'cls' (lấy token đầu tiên) hoặc 'last' (token cuối cùng)
    """
    # Tokenize văn bản
    inputs = tokenizer(
            [
                PROMPT.format(
                    prompt,
                    essay
                )
            ], return_tensors="pt").to("cuda")

    # Lấy các hidden state từ mô hình
    # Lưu ý: Một số mô hình yêu cầu thiết lập config để trả về hidden_states, nếu không thì bạn cần set config.output_hidden_states=True.
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True, return_dict=True)

    # Giả sử sử dụng hidden states từ layer cuối cùng
    hidden_states = outputs.hidden_states[-1]  # shape: (batch_size, seq_len, hidden_size)

    # Thực hiện pooling: ví dụ trung bình các hidden states theo chiều seq_len
    if pooling == 'mean':
        embedding = hidden_states.mean(dim=1)
    elif pooling == 'cls':
        # Nếu mô hình có token [CLS] ở vị trí đầu tiên
        embedding = hidden_states[:, 0, :]
    elif pooling == 'last':
        # Lấy token cuối cùng
        embedding = hidden_states[:, -1, :]
    else:
        raise ValueError("Chọn pooling là 'mean', 'cls' hoặc 'last'.")

    return embedding

# Giả sử DataFrame 'data' chứa các cột 'prompt' và 'essay'
# Ví dụ: data = pd.read_csv("your_data.csv")

# Danh sách lưu embedding cho từng dòng (sẽ lưu dưới dạng danh sách 1D)
embeddings = []

# Nếu data có ít hơn 200 dòng, xử lý toàn bộ; ngược lại chỉ xử lý 200 dòng đầu tiên
for i in tqdm(range(len(data)), desc="Processing embeddings"):
    prompt = data['prompt'][i]
    essay = data['essay'][i]
    combined_emb = get_embedding(prompt, essay)

    # Chuyển thành vector 1D và chuyển sang danh sách để dễ lưu vào CSV
    embeddings.append(combined_emb.flatten().tolist())

# Tạo DataFrame mới chứa cột embedding
df_embeddings = pd.DataFrame({'embedding': embeddings})

# Lưu vào file CSV (mỗi embedding sẽ được lưu dưới dạng chuỗi danh sách)
df_embeddings.to_csv('/content/drive/MyDrive/embeddings_full.csv', index=False)

Processing embeddings:  84%|████████▍ | 8305/9831 [2:29:59<28:29,  1.12s/it]

Lấy embedding

In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 1. Đọc file CSV
df = pd.read_csv('/content/drive/MyDrive/embeddings_full.csv')

# 2. Chuyển cột 'embedding' thành danh sách các mảng NumPy
def str_to_numpy(embedding_str):
    # Chuyển chuỗi thành danh sách số
    embedding_list = literal_eval(embedding_str)
    return np.array(embedding_list)

# Áp dụng hàm để chuyển toàn bộ cột 'embedding' thành mảng NumPy
embeddings = df['embedding'].apply(str_to_numpy).tolist()

# Kiểm tra kích thước một embedding (nên là (3072,))
print("Kích thước của một embedding:", embeddings[0].shape)  # Nên là (3072,)

# Đảm bảo chỉ lấy 10 mẫu đầu tiên (num_samples = 10)
# num_samples = 10
# embeddings = embeddings[:num_samples]  # Lấy 10 mẫu đầu tiên

# Chuyển danh sách embeddings thành mảng 2D với shape (num_samples, 3072)
X = np.array(embeddings)  # Shape: (10, 3072)

# Kiểm tra kích thước của X
print("Kích thước của X:", X.shape)  # Nên là (10, 3072)

In [ ]:
# prompt: In ra các cột của data

data.columns

Index(['Unnamed: 0', 'prompt', 'essay', 'Task Achievement',
       'Coherence and Cohesion', 'Lexical Resource',
       'Grammatical Range and Accuracy', 'embedding'],
      dtype='object')

In [ ]:
# prompt: Xóa đi cột Unnamed: 0 trong data

# Assuming 'data' DataFrame is already loaded and processed as in the provided code.

# Check if the column exists before dropping it
if 'Unnamed: 0' in data.columns:
    data = data.drop(columns=['Unnamed: 0'])

In [ ]:
data.head()

,prompt,essay,Task Achievement,Coherence and Cohesion,Lexical Resource,Grammatical Range and Accuracy,embedding
0,Interviews form the basic criteria for most la...,It is believed by some experts that the tradit...,7.0,7.5,7.0,7.0,NaN
1,Interviews form the basic selecting criteria f...,Nowadays numerous huge firms allocate an inter...,5.0,4.5,4.0,4.5,NaN
2,Interview form the basic selection criteria fo...,The interview section is the most vital part o...,6.5,6.5,6.0,6.0,NaN
3,Interviews form the basic selection criteria f...,It is argued that the best method to recruit e...,6.0,5.5,5.0,5.0,NaN
4,Interviews from the basic selecting criteria f...,Nowadays many companies conduct interviews bef...,3.5,3.0,3.0,3.0,NaN


In [ ]:
# prompt: Lưu data thành file csv

# Lưu DataFrame thành file CSV
data.to_csv('/content/drive/MyDrive/embeddings/ielts_embeddings.csv', index=False)